# 작업형1

## 문제1

주어진 코로나19 데이터를 활용하여 코로나19 사망자가 가장 많이 발생한 나라 의 수치와 사망자가 5번째로 적게 발생한 나라의 수치의 차이를 출력하시오. (단, 중복값은 고려하지 않으며, 특수 문자('-')가 포함된 데이터의 값은 0으로 대체 한다.)

In [1]:
import numpy as np
import pandas as pd 

df = pd.read_csv('/kaggle/input/bigdata-csvfile/covid_death_bycountry.csv')

df['Deaths'] = df['Deaths'].replace('—', 0)
df['Deaths'] = df['Deaths'].astype(int)

df = df.sort_values(by='Deaths', ascending = False)
print(df['Deaths'].iloc[0] - df['Deaths'].iloc[-5])
# 가장 많은 사망자 나라 수치 - 5번째로 적게 발생한 나라의 수치

1110232


## 문제2

다음은 2000년부터 2022년까지 국내의 지역별 출생률, 사망률, 이혼율, 혼인율,
자연 성장률에 대한 데이터이다. 

2008년 7월 1일 데이터 중 출생률이 가장 높은 지역의 이름을 출력하시오. (단, 결측값이 포함된 경우 결측값은 해당 칼럼의 중 앙값으로 대체한다.)

In [2]:
import numpy as np
import pandas as pd 

df = pd.read_csv('/kaggle/input/bigdata-csvfile/Korean_demographics_2000-2022.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

df['Birth_reate'] = df['Birth_rate'].fillna(df['Birth_rate'].median())
cond = df.loc[(df['Year']==2008) & (df['Month'] == 7) & (df['Day'] == 1)]
df = cond.sort_values(by = 'Birth_rate', ascending=False)
print(df['Region'].iloc[0]) # Gyeonggi-do

Gyeonggi-do


## 문제3

주어진 수면 데이터에서 수면시간(Sleep Duration) 칼럼과 가장 높게 상관계수를 갖는 변수의 최빈값을 출력하시오.
(단, Blood Pressure 컬럼과 Sleep Disorder 컬럼은 분석에서 제외한다.)

In [3]:
import numpy as np
import pandas as pd 

df = pd.read_csv('/kaggle/input/bigdata-csvfile/Sleep_health_and_lifestyle_dataset.csv')

# 필요없는 컬럼 제거
df = df.drop(columns = ['Gender','Blood Pressure', 'Sleep Disorder', 'Age', 'Occupation', 'BMI Category'])
corr = df.corr()

# 상관계수 출력
corr_sleep = (corr['Sleep Duration'].drop('Sleep Duration')).sort_values(ascending=False)
print((np.abs(corr_sleep)).idxmax())  #Quality of Sleep : 0.8832130004106182 

# Quality of Sleep의 최빈값
Quality_of_Sleep_mode = df['Quality of Sleep'].mode()[0]
print(Quality_of_Sleep_mode) #최빈값 : 8


Quality of Sleep
8


# 작업형2

1. 전체 데이터 목록 4,000행 중 3,800을 학습용 데이터로 사용하고, 나머지를 테스트 데이터로 사용할 수 있도록 데이터를 슬라이싱한다.
2. 학습용 데이터를 활용하여 예측 모델을 모델링하고, 테스트 데이터를 적용하여 목표 변수를(price) 예측하고, 예측 결과를 제출한다. (단, 제출 결과는 테스트 데이터의 개수인 209행이 될 수 있도록 한다.)
3. 모델 평가지표는 RMSE로 한다.
4. 모델 예측 결과는 price 칼럼을 갖고, 예측 결과를 나타내며, index는 표시하지 않 는다.
5. 모델 예측 결과 파일명은 다음과 같이 하여 제출한다. (파일명 : result.csv)

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/used_cars_price_data.csv')
df['price'] = df['price'].str.replace('$','')
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].astype(int)


train = df.iloc[:3800]
test = df.iloc[3800:]
y = train['price']

train_drop = train.drop(columns='price')
test_drop = test.drop(columns='price')

# 인코딩
train_dummies = pd.get_dummies(train_drop)
test_dummies = pd.get_dummies(test_drop)

for col in train_dummies.columns:
    if col not in test_dummies.columns:
        train_dummies.drop(columns=col, axis=1, inplace=True)

for col in test_dummies.columns:
    if col not in train_dummies.columns:
        test_dummies.drop(columns=col, axis=1, inplace=True)

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train_dummies, y)
pred = model.predict(test_dummies)

# DataFrame생성
result = pd.DataFrame({'pred':pred})
# csv파일 생성
result.to_csv('result.csv', index=False)
result_csv = pd.read_csv('result.csv')
print(result_csv)

           pred
0     71041.810
1     19205.025
2     14877.500
3     46347.150
4     23399.600
..          ...
204  224518.200
205   65704.320
206  162349.260
207   51974.870
208   44375.190

[209 rows x 1 columns]


# 작업형 3

## 문제1
제공되는 심혈관 질한데이터를 활용하여 심혈관 질환 발생 여부(cardio)를 예측 하고자 한다.
각 문항의 답을 제출 형식에 맞게 제출하시오

1. alco, cardio 변수 간의 독립성 검정을 실시할 때, 카이제곱 통계량을 구하시오.
2. gender, weight, smoke, cholesterol을 독립변수로 사용하여 로지스틱 회귀 모형 으로 분석할 때, smoke 변수의 계수 값을 구하시오.
3. 2번 문제에서 생성한 로지스틱 회귀 모형에서 gender 변수가 한 단위 증가할 때 심 혈관 질환이 발생할 오즈비의 값을 구하시오. (단, 모든 문제의 정답은 반올림하여 소수점 넷째 자리까지 출력한다.)

- id : 측정자 id
- age : 나이
- gender : 성별
- height : 키
- ap_hi : 수축 혈압
- ap_lo : 이완 혈압
- cholestrol : 콜레스테롤 (1:평균, 2:높음, 3:매우 높음)
- glue : 포도당 (1:평균, 2:높음, 3:매우 높음)
- smoke : 흡연여부
- alco : 음주여부
- active : 신체 활동
- cardio : 심혈관 질환 여부 (목표변수)

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bigdata-csvfile/cardiovascular_heart_disease_data.csv')

contingency = pd.crosstab(df['alco'], df['cardio'])
# 1. alco, cardio 변수 간의 독립성 검정을 실시할 때, 카이제곱 통계량을 구하시오.
from scipy import stats
st, pv, dof, exp_f = stats.chi2_contingency(contingency) 
print(round(st,4)) # 3.6965

# 2.gender, weight, smoke, cholesterol을 독립변수로 사용하여 로지스틱 회귀 모형 으로 분석할 때, 
# smoke 변수의 계수 값을 구하시오.
from statsmodels.formula.api import logit
formula = ('cardio ~ gender + weight + smoke + cholesterol')
model = logit(formula, data=df).fit()
# print(model.summary()) # 전체 통계표
print(round(model.params['smoke'],4)) #-0.2174

# 2번 문제에서 생성한 로지스틱 회귀 모형에서 gender 변수가 한 단위 증가할 때 심 혈관 질환이 발생할 오즈비의 값을 구하시오. 
# (단, 모든 문제의 정답은 반올림하여 소수점 넷째 자리까지 출력한다.)
odds_ratio = np.exp(model.params['gender'])
print(round(odds_ratio,4)) #1.0002

3.6965
Optimization terminated successfully.
         Current function value: 0.654936
         Iterations 5
-0.2174
1.0002


## 문제2

1. 독립변수를 exang, trestbps, ca로 하고, 목표변수를 target으로 하는 로지스틱 회 귀 분석을 진행했을 때, ca 변수의 표준오차를 구하시오.
2. fbs, thalach, chol, sex를 독립변수로 target을 분류하는 로지스틱 회귀 분석 모 형을 생성했을 때, 학습(train) 데이터의 오분류율을 구하시오. (단, 전체 데이터 중
800행을 학습(train) 데이터로 사용하고, train test_split의 test_size는 0.2로 하 며, random_state는 123으로 한다.)
3. 2번 문제 모델의 심장 질환 발생(1)에 대한 정밀도를 구하시오. (단, 모든 문제의 정 답은 반올림하여 소수점 둘째 자리까지 출력한다.)

In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/bigdata-csvfile/heart_disease_data.csv')

# 1. ca변수의 표준오차 std err
from statsmodels.formula.api import logit
formula = ('target ~ exang + trestbps + ca')
model = logit(formula, data=df).fit()
# print(model.summary()) 0.083

# 2 오류분율 = 1 - 정확도
from sklearn.model_selection import train_test_split

df_sub = df[['fbs', 'thalach', 'chol', 'sex', 'target']][:800]
train, test = train_test_split(df_sub, test_size = 0.2, random_state=123)

# formula방식
model = logit('target ~ fbs + thalach + chol + sex', data=train).fit()

# 예측
pred = model.predict(test)
pred = (pred > 0.5).astype(int)

# 정확도 계산
accuracy = (pred == test['target']).mean()
# 오류분율 = 1 - 정확도
print(round(1 - accuracy, 2)) 

# 3. 정밀도 구하기
from sklearn.metrics import precision_score
precision = precision_score(test['target'], pred)
print(round(precision,2))

Optimization terminated successfully.
         Current function value: 0.516009
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.520073
         Iterations 6
0.29
0.74
